# Group expenses

In [1]:
%pip install 'opvious>=0.16.0'

## Problem formulation

In [2]:
import opvious.modeling as om

class GroupExpenses(om.Model):
    """A mixed-integer model for finding the optimal way to settle debts within a group"""
    
    members = om.Dimension()  # Participants
    transactions = om.Dimension()  # Expenses
    payment = om.Parameter.non_negative(transactions, members)  # Amount paid so far by each member
    min_transfer = om.Parameter.non_negative()  # Minimum transfer threshold
    
    def __init__(self):
        super().__init__()
        # Additional amount to be paid from one member to another to achieve fairness
        self.transfer = om.Variable.non_negative(self.members, self.members, qualifiers=['sender', 'recipient'])
        # Indicator variable representing a transfer from one member to another (1 if transfer > 0, 0 otherwise)
        self.is_transferring = om.fragments.ActivationVariable(self.transfer, upper_bound=self.total_payments())
    
    def total_payments(self):
        """Total amount of money paid, used as upper-bound for transfers"""
        return om.total(self.payment(t, m) for t, m in self.transactions * self.members)
    
    def fair_payment(self, t):
        """Fair payment per member for a transaction"""
        return om.total(self.payment(t, m) for m in self.members) / om.size(self.members)
    
    @om.constraint
    def zero_sum_transfers(self):
        """After netting transfers, each member should have paid the sum of their fair payments"""
        for m in self.members:
            received = om.total(self.transfer(s, m) for s in self.members)
            sent = om.total(self.transfer(m, r) for r in self.members)
            owed = om.total(self.payment(t, m) - self.fair_payment(t) for t in self.transactions)
            yield received - sent == owed
            
    @om.constraint
    def transfer_meets_min(self): 
        """Each positive transfer is at least equal to the minimum threshold"""
        for s, r in self.members * self.members:
            yield self.transfer(s, r) >= self.is_transferring(s, r) * self.min_transfer()
            
    @om.objective
    def minimize_total_transferred(self):
        """First objective: minimize the total amount of money transferred between members"""
        return om.total(self.transfer(s, r) for s, r in self.members * self.members)
    
    @om.fragments.magnitude_variable(members, projection=0, lower_bound=False)
    def max_transfers_sent(self, m):
        """Number of transfers sent by each member"""
        return om.total(self.is_transferring(m, r) for r in self.members)
    
    @om.objective
    def minimize_max_transfers_sent(self):
        """Second objective: minimize the maximum number of transfers sent by any member"""
        return self.max_transfers_sent()


model = GroupExpenses()
model.definition_counts().T # Summary of the model's components

title,GroupExpenses
category,
CONSTRAINT,4
DIMENSION,2
OBJECTIVE,2
PARAMETER,2
VARIABLE,3


For the mathematically inclined, models can generate their LaTeX specification.

In [3]:
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">GroupExpenses</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^v_\mathrm{transfer[sender,recipient]}&: \tau \in \mathbb{R}_+^{M \times M} \\
  \S^v_\mathrm{isTransferring}&: \tau^\mathrm{is} \in \{0, 1\}^{M \times M} \\
  \S^c_\mathrm{isTransferringActivates}&: \forall m, m' \in M, \sum_{t \in T, m'' \in M} p_{t,m''} \tau^\mathrm{is}_{m,m'} \geq \tau_{m,m'} \\
  \S^d_\mathrm{members}&: M \\
  \S^d_\mathrm{transactions}&: T \\
  \S^p_\mathrm{payment}&: p \in \mathbb{R}_+^{T \times M} \\
  \S^p_\mathrm{minTransfer}&: t^\mathrm{min} \in \mathbb{R}_+ \\
  \S^c_\mathrm{zeroSumTransfers}&: \forall m \in M, \sum_{m' \in M} \tau_{m',m} - \sum_{m' \in M} \tau_{m,m'} = \sum_{t \in T} \left(p_{t,m} - \frac{\sum_{m' \in M} p_{t,m'}}{\# M}\right) \\
  \S^c_\mathrm{transferMeetsMin}&: \forall m, m' \in M, \tau_{m,m'} \geq \tau^\mathrm{is}_{m,m'} t^\mathrm{min} \\
  \S^o_\mathrm{minimizeTotalTransferred}&: \min \sum_{m, m' \in M} \tau_{m,m'} \\
  \S^v_\mathrm{maxTransfersSent}&: \sigma^\mathrm{maxTransfers} \in \mathbb{R}_+ \\
  \S^c_\mathrm{maxTransfersSentUpperBounds}&: \forall m \in M, \sigma^\mathrm{maxTransfers} \geq \sum_{m' \in M} \tau^\mathrm{is}_{m,m'} \\
  \S^o_\mathrm{minimizeMaxTransfersSent}&: \min \sigma^\mathrm{maxTransfers} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

In [4]:
import opvious

_client = opvious.Client.default()

async def optimal_transfers(payments, min_transfer=0, tolerance=0.1):
    """Compute optimal transfers"""
    res = await _client.run_solve(
        specification=model.specification(),
        parameters={'payment': payments.stack()['payment'], 'minTransfer': min_transfer},
        strategy=opvious.SolveStrategy(  # Multi-objective strategy
            epsilon_constraints=[
                # Within tolerance of the smallest total transfer amount
                opvious.EpsilonConstraint('minimizeTotalTransferred', relative_tolerance=tolerance),
                # Using the smallest possible number of transfers per member
                opvious.EpsilonConstraint('minimizeMaxTransfersSent'),
            ],
            target='minimizeTotalTransferred',  # Final target: minimize total transfer amount
        ),
        assert_feasible=True,
    )
    return res.outputs.variable('transfer').unstack(level=1).fillna(0)

## Testing

In [5]:
import numpy as np
import pandas as pd

_names = ["emma", "noah", "ava", "liam", "isabella", "sophia", "mason", "mia", "lucas", "amelia"]

def random_data(count=15, seed=16):
    rng = np.random.default_rng(seed)
    tuples = []
    for i in range(count):
        tid = f't{i+1:02}'
        for name in _names:
            if rng.integers(2):
                continue
            tuples.append({
                'transaction': tid, 
                'name': name, 
                'payment': round(100 * rng.random(), 2),
            })
    df = pd.DataFrame(tuples).set_index(['transaction', 'name'])
    return df.unstack().fillna(0)

df = random_data()
df

payment                                                            \
name         amelia    ava   emma isabella   liam  lucas  mason    mia   noah   
transaction                                                                     
t01           80.24   0.00   0.00     0.00   9.41   0.00  87.46  85.40   0.00   
t02            0.00  95.86  69.56    66.33   0.00  29.93  39.49  27.80  15.50   
t03            0.00  13.87   0.00    41.36  38.47   0.00   0.00   0.00   0.00   
t04            0.00   0.00   0.00    14.89   0.00  96.19  86.77   0.00  72.99   
t05            0.00   0.00   0.00     0.00   5.80   0.00   0.00   0.00   0.00   
t06           31.91  98.92  94.57    76.46  73.58   0.00  66.66   0.00  38.71   
t07           16.09  20.13   0.00    61.43  19.68   0.00  95.59   0.00   0.00   
t08           53.80  65.37   0.00    87.86   7.57   8.07   0.00  92.50   0.00   
t09            0.00  30.14   8.33     0.00   0.00  58.71   0.00  71.64   0.00   
t10            0.00  85.21  80.83     0.00   0.00  51.47   0.00  98.71  31.98   
t11           62.75  43.65  30.62    66.43  30.35  69.67  62.13  40.25   0.00   
t12            0.00  82.03  32.63     0.00   0.00   5.82   0.00   0.00   0.00   
t13           95.32  94.01  45.25     0.00   0.00  72.72   0.00   0.00   0.00   
t14            0.00  43.56  32.75     0.00   0.00   0.00   6.34   5.46   0.00   
t15           72.38   0.00  65.15     7.22   0.00  60.43  88.12  68.27  28.18   

                    
name        sophia  
transaction         
t01          62.15  
t02           0.00  
t03           0.00  
t04           0.00  
t05           0.00  
t06           0.00  
t07          50.31  
t08          43.37  
t09           0.00  
t10           0.00  
t11          47.44  
t12          35.52  
t13          84.39  
t14          70.42  
t15           0.00

In [6]:
odf = await optimal_transfers(df, min_transfer=10, tolerance=0)
odf

value                                                  
recipient amelia      ava    emma isabella   lucas   mason     mia
sender                                                            
amelia       0.0    0.000   0.000    0.000   0.000  18.343   0.000
isabella    10.0    0.000   0.000    0.000   0.000   0.000   0.000
liam         0.0  150.685   0.000    0.000  32.177   0.000  53.111
noah         0.0  101.232  38.857    0.000   0.000  93.384   0.000
sophia       0.0    0.000   0.000   11.147   0.000   0.000  16.086

Next steps:

+ Partial group transactions (formulated [here](https://github.com/opvious/examples/blob/main/sources/group-expenses.md) for example)
+ Different objective ordering
+ 